In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from matplotlib.axes import Axes
from matplotlib.figure import Figure

import utilities_plot as up
import utilities_latent as ul
from utilities_base import VAE, load_data

In [2]:
data, data_id_dict, child_id_dict, word_dict, category_dict = load_data(
    ["data", "data_id_dict", "child_id_dict", "word_dict", "category_dict"]
)
word_count = len(word_dict)


model = VAE().to("cuda")
model.eval()
model.load_state_dict(torch.load("tmp/best_model.pth"))
# model.load_state_dict(torch.load("tmp/model_state_dict.pth"))

figs = {}

In [3]:
# # 潜在空間を動く点Zsを指定

# x1, x2 = 2, 2
# y1, y2 = 2, -3
# n = 50
# Xs = np.linspace(x1, x2, n)
# Ys = np.linspace(y1, y2, n)
# Zs = np.stack((Xs, Ys), axis=1)

# # 潜在空間を動く点Z1s, Z2sを指定
# X1s = np.linspace(0, 0, n)
# Y1s = np.linspace(2, -3, n)
# Z1s = np.stack((X1s, Y1s), axis=1)
# X2s = np.ones(n) * 2
# Y2s = np.ones(n) * 0
# Z2s = np.stack((X2s, Y2s), axis=1)

In [4]:
# スプライン補完のための点群を指定

xy = [
    (2, 2.5),
    (-1, 2),
    (2, 1.5),
    (-2, 1),
    (2, 0.5),
    (-3, 0),
    (2, -0.5),
    (-2, -1),
    (2, -1.5),
    (-1, -2),
    (2, -2.5),
    (5, 0),
    (2, 2.5),
]

n = 200
Zs = ul.spline_interpolation(xy, n)

In [8]:
# ファイル名の設定
tmp_image_folder = Path("images/tmp")
tmp_expectation_image_name = "expectation"
tmp_point_image_name = "point"
output_gif_path = Path("images/gif/expectation.gif")

# gifにするファイルを選択
image_expectation_dirs1 = [
    str(tmp_image_folder / tmp_expectation_image_name) + f"_{i}.png" for i in range(n)
]
image_expectation_dirs2 = [
    str(tmp_image_folder / tmp_point_image_name) + f"_{i}.png" for i in range(n)
]

In [ ]:
# 点Zsにおけるのカテゴリーごとの期待値を計算しプロット
# そのときのZsの位置もプロット
# gifファイル作成に備えて保存

ul.save_expectation_plot_with_category(
    model, Zs, tmp_image_folder, tmp_expectation_image_name
)
ul.save_zs_plot(model, Zs, tmp_image_folder, tmp_point_image_name)

In [9]:
# gifにして保存
ul.make_combined_gif(image_expectation_dirs1, image_expectation_dirs2, output_gif_path)

In [ ]:
# # Z1s, Z2sの期待値の差をカテゴリーごとにプロット
# # そのときのZ1s, Z2sの位置もプロット
# # gifファイル作成に備えて保存

# ul.save_expectation_diff_plot_with_category(model, Z1s, Z2s, "A", "B", Path("images/tmp/"))
# ul.save_zs_diff_plot(model, Z1s, Z2s, "A", "B", Path("images/tmp/"))

# # gifにするファイルを選択
# image_diff_dirs1 = [f"images/tmp/expectation_diff_{i}.png" for i in range(n)]
# image_diff_dirs2 = [f"images/tmp/point_diff_{i}.png" for i in range(n)]

# # gifにして保存
# ul.make_combined_gif(image_diff_dirs1, image_diff_dirs2, Path("images/gif/diff.gif"))

/workspaces/wordbank/utilities_latent.py:129: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  figs[f"expectation_diff_{i}"] = plt.subplots()


In [ ]:
# アルファベットとカテゴリーの対応関係
df_category = pd.DataFrame(
    {"category": list(category_dict.keys())}, index=[chr(65 + i) for i in range(22)]
)
df_category

,category
A,sounds
B,animals
C,vehicles
D,toys
E,food_drink
F,clothing
G,body_parts
H,household
I,furniture_rooms
J,outside
